In [52]:
import os, json, getpass, asyncio, logging, requests, inspect
from typing import Literal

# from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chat_models import init_chat_model

from langgraph.types import Command
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_community.tools import DuckDuckGoSearchRun

from langgraph.prebuilt import create_react_agent
# from langgraph_supervisor import create_supervisor
from langchain.tools import Tool

from fastmcp import Client

In [53]:
# Env
from dotenv import load_dotenv
_ = load_dotenv()

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

In [54]:
os.getenv("GOOGLE_API_KEY")
gmodel = init_chat_model("google_genai:gemini-2.0-flash")
gchat = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [55]:

async def _search_data_async(question: str, limit: int = 8) -> str:
    async with Client("http://127.0.0.1:8050/mcp-server/mcp") as c:
        res = await c.call_tool("search_data", {"query": question, "limit": limit})
        return json.loads(res)["answer"]

In [56]:
from langchain_core.tools import tool, ToolException
from functools import partial
from functools import wraps
from fastmcp.utilities.types import TextContent
from mcp.server.fastmcp import Context

In [57]:
# def _sync_bridge(coro, *args, **kwargs):
#     """Exécute une coroutine dans la boucle déjà active (Jupyter) et renvoie le résultat."""
#     loop = asyncio.get_running_loop()
#     fut  = asyncio.run_coroutine_threadsafe(coro(*args, **kwargs), loop)
#     return fut.result()  

def sync_bridge(coro_fn): 
    """
    Décorateur : transforme une coroutine en fonction bloquante
    qui s'exécute indifféremment dans un thread Jupyter ou non.
    """
    @wraps(coro_fn)
    def _wrapper(*args, **kwargs):
        try:
            # ⓵ Cas Jupyter ou event-loop déjà active
            loop = asyncio.get_running_loop()
        except RuntimeError:
            loop = None

        if loop and loop.is_running():
            # On poste la coroutine dans la boucle existante
            fut = asyncio.run_coroutine_threadsafe(coro_fn(*args, **kwargs), loop)
            return fut.result()
        else:
            # ⓶ Thread sans boucle → on en crée une locale
            return asyncio.run(coro_fn(*args, **kwargs))
    _wrapper.__signature__ = inspect.signature(coro_fn)
    return _wrapper         # bloque cette cellule - OK pour un tool

In [58]:
async def _call_search(query: str, limit: int):
    async with Client("http://127.0.0.1:8050/mcp-server/mcp") as c:
        res = await c.call_tool("search_data", {"ctx": {}, "query": query, "limit": limit})

    # ── normaliser ─────────────────────────────
    if isinstance(res, TextContent):
        payload = res.text
    elif isinstance(res, (bytes, str)):
        payload = res
    elif isinstance(res, list) and res and isinstance(res[0], TextContent):
        payload = res[0].text
    else:
        raise ValueError(f"Format inattendu: {type(res)}")

    data = json.loads(payload)      # <─ plus de TypeError
    return data["answer"]

In [59]:
# Le LLM ignore toujours nos paramètres → LangChain ne lui a pas transmis le JSON schema.
# Il faut imposer le schéma via le décorateur @tool qui crée d’office un objet BaseTool.
# Remarque : avec @tool, pas besoin de Tool.from_function ni d’args_schema.
# LangChain génère automatiquement la spec OpenAPI (et Gemini la respectera).

@tool(
    # name        = "RealEstateRAGTool",
    description = "Recherche projets et biens immobiliers.",
)
@sync_bridge # <- garde la compatibilité sync
async def realestate_rag(question: str, limit: int = 20) -> str: # _search_data
    """Interroge GraphRAG : retourne une réponse textuelle."""
    try:
        return await _call_search(question, limit)
    except ToolException as e:
        return f"[ERREUR RAG] {e}"

In [34]:
# rag_tool = tool.from_function(
#     name        = "RealEstateRAGTool",
#     description = "Recherche projets et biens immobiliers (GraphRAG).",
#     func        = realestate_rag,
#     return_direct = True        # l'agent renvoie directement la réponse du tool
# )

In [60]:
# -----------------------------
# InfoAgent (RAG-based agent for project/property info)
# -----------------------------
info_agent = create_react_agent(
    model=gchat,
    tools=[realestate_rag],  # rag_tool : Replace with my custom RAG retriever tools
    prompt=(
        "You are the InfoAgent.\n"
        "- Answer questions about real estate projects, properties, availability, location, prices, and financial conditions.\n"
        "- Use only the available rag_tool to query property databases or retrieve knowledge from the RAG system.\n"
        "- You may also explain financial aspects such as payment options or loan eligibility (just based on information from the tool).\n"
        "- Do NOT collect client personal information.\n"
        "- Do NOT schedule appointments."
        "- Do Not give any external information from the llm model, only use the rag_tool to answer questions.\n"
        "- If you don't know the answer, say 'I don't know' or 'I cannot answer that'.\n"
    ),
    name="InfoAgent"
)

────────────────────────────────
**Utilisation notebook**
────────────────────────────────

In [61]:
# 🟢 Appel unique
resp = info_agent.invoke(
    # {"messages":[{"role":"user","content":"Quel est le prix des F2 à Bouskoura ?"}]}
    {"messages":[{"role":"user","content":"Y a t-il des laucaux commerciaux à Bouskoura ? Peut-on trouver des locaux commerciaux à moins de 500 000 MAD ?"}]}
)
print(resp)               # dict complet
print(resp["messages"][-1].content)  # texte final

{'messages': [HumanMessage(content='Y a t-il des laucaux commerciaux à Bouskoura ? Peut-on trouver des locaux commerciaux à moins de 500 000 MAD ?', additional_kwargs={}, response_metadata={}, id='953b4e48-9f80-42e8-94e8-478a89ba1a7d'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'realestate_rag', 'arguments': '{"limit": 5.0, "question": "locaux commerciaux \\u00e0 Bouskoura \\u00e0 moins de 500 000 MAD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, name='InfoAgent', id='run--a5e0a1ef-c766-46a8-be32-9243b0271525-0', tool_calls=[{'name': 'realestate_rag', 'args': {'limit': 5.0, 'question': 'locaux commerciaux à Bouskoura à moins de 500 000 MAD'}, 'id': 'cf6378d3-4eec-4cd7-a323-4465805a400c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 189, 'output_tokens': 26, 'total_tokens': 215, 'input_token_details': {'cache_read': 0}}), ToolMessa

In [62]:
# 🟢 Streaming
for chunk in info_agent.stream(
    # {"messages":[{"role":"user","content":"Quel est le prix des F3 à Bouskoura ?"}]}
    # {"messages":[{"role":"user","content":"Quels biens avez-vous à vendre à Casablanca ?"}]}
    {"messages":[{"role":"user","content":"Dannez moi la liste des projets situés à Casablanca ?"}]}
):
    print(chunk)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'realestate_rag', 'arguments': '{"limit": 5.0, "question": "projets situ\\u00e9s \\u00e0 Casablanca"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, name='InfoAgent', id='run--9729ceb4-a50c-4c4c-bbe5-7024c33c9f8d-0', tool_calls=[{'name': 'realestate_rag', 'args': {'limit': 5.0, 'question': 'projets situés à Casablanca'}, 'id': '2d496970-bec2-433f-a0e9-b7492dbd518b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 166, 'output_tokens': 12, 'total_tokens': 178, 'input_token_details': {'cache_read': 0}})]}}
{'tools': {'messages': [ToolMessage(content='À Casablanca, le Groupe Addoha propose les projets suivants\xa0:\n\n* **AL FATH (Route Arrahma):** Appartements F3 économiques à partir de 200\xa0000 dhs.  Le projet inclut une école, une crèche, une mosquée, un centre de santé